In [ ]:
%pip install openai tiktoken chromadb langchain

# 使用MapReduce的方式来提炼《神雕侠侣》

In [28]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ReduceDocumentsChain, MapReduceDocumentsChain
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import UnstructuredEPubLoader
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chat_models import ChatOpenAI
import time

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",openai_api_base="http://3.101.17.202/v1")

# Map
map_template = """The following is a set of documents
{docs}
Based on this list of docs, please identify the main themes 
Helpful Answer:"""
map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """The following is set of summaries:
{doc_summaries}
Take these and distill it into a final, consolidated summary of the main themes in Simplfied Chinese. 
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm,prompt=reduce_prompt)
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries"
)

# 生成摘要（递归合并）
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=16000,
)

# Map - Reduce
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="docs"
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=12000, chunk_overlap=0 # 注意：不同于知识库，overlap一定要是0，不然会有重复内容。
)

# loader = UnstructuredEPubLoader('shendiao.epub')
loader = WebBaseLoader("https://www.marxists.org/chinese/marx/01.htm")
docs = loader.load()
split_docs = text_splitter.split_documents(docs)

map_reduce_chain.run(split_docs)
print( map_reduce_chain.run(split_docs))


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).


根据提供的文件列表，可以总结出以下主题：

1. 批判资本主义：文件讨论了对资本主义的批判，强调了资本主义制度的剥削性质和不公平性。

2. 阶级斗争：文件探讨了资产阶级（资本家阶级）和无产阶级（工人阶级）之间的阶级斗争，强调了工人阶级团结起来推翻资产阶级的必要性。

3. 社会主义和共产主义：这些文件讨论了社会主义和共产主义作为替代资本主义的理论和实践方面。它们探讨了这些意识形态的原则和目标，以及它们创造更公平和正义社会的潜力。

4. 对立政党的态度：《共产党宣言》还讨论了共产主义者对各种反对派的立场。它讨论了共产主义者与其他政治团体之间的关系，并强调了工人阶级团结的必要性。

总的来说，主要主题围绕对资本主义的批判、社会阶级斗争、社会主义和共产主义的目标，以及共产党在为工人阶级的权益和利益辩护方面的作用。


### 使用 Refine 方式来提取

In [29]:
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.document_loaders import WebBaseLoader
from langchain.document_loaders import UnstructuredEPubLoader

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-16k",openai_api_base="http://3.101.17.202/v1")
prompt_template = """Write a concise summary of the following:
{text}
CONCISE SUMMARY:"""
prompt = PromptTemplate.from_template(prompt_template)

refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n" 
    "------------\n"
    "Given the new context, refine the original summary in Sinplfied Chinese."
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate.from_template(refine_template)
chain = load_summarize_chain(
    llm=llm,
    chain_type="refine",
    question_prompt=prompt,
    refine_prompt=refine_prompt,
    input_key="input_documents",
    output_key="output_text",
)

loader = WebBaseLoader("https://www.marxists.org/chinese/marx/01.htm")
docs = loader.load()
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=12000, chunk_overlap=0 # 注意：不同于知识库，overlap一定要是0，不然会有重复内容。
)
split_docs = text_splitter.split_documents(docs)

result = chain({"input_documents": split_docs}, return_only_outputs=True)
print(result['output_text'])

根据新的背景信息，无产阶级的早期运动失败是因为无产阶级本身还不够发展，无产阶级解放的物质条件还没有具备。早期的社会主义和共产主义体系是在无产阶级和资产阶级之间的斗争还不发展的最初时期出现的，这些体系的创始人看不到无产阶级的历史主动性和政治运动，只能通过幻想的条件来创造无产阶级解放的物质条件。这些体系的创始人虽然意识到阶级对立和社会瓦解的作用，但他们的主张仍带有纯粹空想的性质。批判的空想的社会主义和共产主义逐渐失去实践意义和理论根据，他们的信徒也变得反动和保守。共产党人支持各种反对现存社会制度和政治制度的革命运动，并强调所有制问题是运动的基本问题。他们努力争取全世界民主政党之间的团结和协调，并公开宣布只有通过暴力推翻现存社会制度才能实现他们的目的。《共产党宣言》在国际工人运动中起到了重要的作用，它是无产阶级运动的纲领，被广泛传播并被工人阶级公认。此外，还有一些其他的空想社会主义者和共产主义者，如英国的欧文派、法国的傅立叶派、埃・卡贝和魏特林，他们的理论和实践也都有一定的局限性和失败的原因。根据新的背景信息，雅科比是德国医生，共产主义者同盟盟员和特使，他在科隆共产党人案件中被控“侮辱国王陛下”并被监禁，后流亡英国并迁居美国，在美国宣传马克思主义思想。亚历山大三世是俄国皇帝。


可能那种靠正序时间线推进的小说类文章更适合使用 Refine 的模式。